In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
from scipy.stats import norm,multivariate_normal

import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import torch
from torchvision import transforms



In [56]:
# logging_index
# eval_index

# logging_policy 1
# eval_policy 1
# logging_xshift 1
# eval_xshift 1
# first_component 1
# logging_xy 1
# eval_xy 1
# logging_reward 1
# eval_reward 1
# logging_label 1
# eval_reward 1



In [78]:
# box 1

# xy
# label
# fc

path = 'training.pt'
features_, labels = torch.load(path)

trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

features = torch.zeros((len(features_), 10, 10))
for i in range(len(features_)):
    features[i] = trans(features_[i])

features = features.numpy().reshape(-1, 10 * 10)
labels = labels.numpy()

xy = features
label = labels

n_class = 10
n_dim = 10*10




n_class = 10
n_dim = xy.shape[1]

train_xy = xy[:int(len(xy)*0.7)]
test_xy = xy[int(len(xy)*0.7):]

train_label = label[:int(len(xy)*0.7)]
test_label = label[int(len(xy)*0.7):]




pca_model = PCA(n_components=1).fit(xy)
fc = pca_model.transform(xy)

train_fc = fc[:int(len(xy)*0.7)]
test_fc = fc[int(len(xy)*0.7):]

train_size = int(len(xy)*0.7)



In [79]:
# box 2

logging_a = 0.8
logging_b = 0.3

eval_a = 1
eval_b = 0.01

def get_distribution(fc,a,b,pca_model=None):
    m = np.min(fc)
    m_bar = np.mean(fc)
    mean = m + (m_bar-m)/a
    std = np.sqrt((m_bar - m)/b)
#     pdf = norm(loc = mean,scale = std).pdf(fc)
    pdf = np.ones(len(fc))
    return pdf/sum(pdf), mean, std








In [80]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_soften(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(train_xy[:int(train_size*0.1)], train_label[:int(train_size*0.1)])
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = alpha + u*beta
            else:
                tmp[0,t] = (1-alpha - u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(train_xy, train_label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [81]:
# # box 3
# # logging policy, eval_policy
# logging_policy_interval = [0.5,0.8]
# eval_policy_interval = [0.8,0.9]

def generate_policy_adv(alpha,beta):
    clf_policy = LogisticRegression(random_state=0).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    logging_policy = np.zeros((len(xy),n_class))

    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
        u = np.random.uniform(-0.5,0.5)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 1- alpha - u*beta
            else:
                tmp[0,t] = (alpha + u*beta)/(n_class-1)
        logging_policy[i] = tmp



    clf_policy = LogisticRegression(random_state=1).fit(xy, label)
    predict_label = clf_policy.predict(xy)
    eval_policy = np.zeros((len(xy),n_class))
    p_max = []
    for i in range(len(xy)):
        tmp = np.zeros((1,n_class))
#         u = np.random.uniform(-0.05,0.05)
        for t in range(n_class):
            if t == predict_label[i]:
                tmp[0,t] = 0.9
            else:
                tmp[0,t] = (1-0.9)/(n_class-1)
        eval_policy[i] = tmp
    return logging_policy, eval_policy
    

In [82]:
# sample data
# just sample index and everything can be refered from index.
def take_action(policy):
    action = random.choices([t for t in range(n_class)],policy)[0]
    prob = policy[action]
    return action, prob


def sample_logging_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    logging_index = np.zeros(size,np.int)
    logging_action = np.zeros(size,np.int)
    logging_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
        t_index = i
        logging_index[i] = t_index
        
        action,_ = take_action(logging_policy[t_index])
        logging_action[i] = action
        
        logging_reward[i] = int(action == label[t_index])
    
    return logging_index, logging_action, logging_reward




def sample_eval_data(xy, label, size, pdf, pdf_eval, logging_policy, eval_policy):
    eval_index = np.zeros(size,np.int)
    eval_action = np.zeros(size,np.int)
    eval_reward = np.zeros(size,np.int)
    index_list = [i for i in range(len(xy))]
    
    for i in range(size):
        t_index = random.choices(index_list,weights=np.ones(len(xy)))[0]
        t_index = t_index  + train_size
        eval_index[i] = t_index  
        
        action,_ = take_action(logging_policy[t_index])
        eval_action[i] = action
        
        eval_reward[i] = int(action == label[t_index-train_size])
    
    return eval_index , eval_action, eval_reward




        



In [83]:
def p_xshift(train_x,test_x):
    from sklearn.linear_model import LogisticRegression
    X = np.concatenate((train_x,test_x),axis = 0)
#     X = np.sum(X,axis = 1,keepdims = True)
    Y = np.concatenate((np.ones(len(train_x)),np.zeros(len(test_x))),axis = 0)
    clf = LogisticRegression().fit(X,Y)
    return clf

def loggingpolicy(x,y):
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression().fit(x,y)
    return clf

In [84]:
def evaluate_ips(logging_index, logging_action, logging_reward,\
                 eval_index, eval_action, eval_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT):
    
    
    
    
#     GT = np.mean(eval_reward)
    ips_list = []
    snips_list = []
    ips_r_list = []
    
    ips_list_unknown = []
    snips_list_unknown = []
    ips_r_list_unknown = []
    
    
    
    for k in range(1):
#         total_reward = 0
#         for i in range(len(eval_index)):
#             action, prob = take_action(eval_policy[eval_index[i]])
#             if action == label[eval_index[i]]:
#                 total_reward += 1
#         GT = total_reward/len(eval_index)

        
    #     known:

        p_w = my_bound(eval_policy[eval_index, eval_action]/logging_policy[eval_index, eval_action])
#         x_w = pdf_eval[logging_index]/pdf[logging_index]
        x_w = 1
        
        

        ips = p_w*eval_reward
        ips_loss = (np.mean(ips) - GT)**2
        ips_list.append(ips_loss)

        snips = p_w*eval_reward/np.mean(p_w)
        snips_loss = (np.mean(snips) - GT)**2
        snips_list.append(snips_loss)

        ips_r = x_w*p_w*eval_reward
        ips_r_loss = (np.mean(ips_r) - GT)**2
        ips_r_list.append(ips_r_loss)




    #     unknown:
        clf_xshift = p_xshift(xy[logging_index],xy[eval_index])
        clf_logging = loggingpolicy(xy[logging_index],logging_action)


        logging_predicted = clf_logging.predict_proba(xy[eval_index])[np.arange(len(eval_index)),np.array(eval_action,np.int)]
        p_w = eval_policy[eval_reward, eval_action]/logging_predicted

        xshift_logging = clf_xshift.predict_proba(xy[eval_reward])
        x_w = xshift_logging[:,1]/xshift_logging[:,0]



        ips = p_w*eval_reward
        ips_loss_unknown = (np.mean(ips) - GT)**2
        ips_list_unknown.append(ips_loss_unknown)

        snips = p_w*eval_reward/np.mean(p_w)
        snips_loss_unknown = (np.mean(snips) - GT)**2
        snips_list_unknown.append(snips_loss_unknown)

        ips_r = x_w*p_w*eval_reward
        ips_r_loss_unknown = (np.mean(ips_r) - GT)**2
        ips_r_list_unknown.append(ips_r_loss_unknown)

    
    return np.mean(ips_list),np.mean(snips_list),np.mean(ips_r_list),\
                np.mean(ips_list_unknown),np.mean(snips_list_unknown),np.mean(ips_r_list_unknown),
    
    

    
    
    
    
    
    
    
    
    

In [85]:
from ope_v3 import *


class WEIGHT_DATA_SET(data.Dataset):
    '''
    dataset class with instance weight 
    '''
    def __init__(self, data,reward, weights):
        '''
        weights are same dimensional witi original data
        '''
        self.data = torch.tensor(data)
        self.reward = torch.tensor(reward)
        self.weights = torch.tensor(weights)

    def __getitem__(self, index):
        img = self.data[index] 
        target = self.reward[index]
        weight = self.weights[index]
        return img, target, weight

    def __len__(self):
        return len(self.data)

# r is xshift weight
# w is policy shift weight
def train_robust_regression(logging_index, logging_action, logging_reward,\
                 eval_index, eval_action, eval_reward,\
                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                            dm=False,xshift_known = True, policy_known = True,use_xshift = True):
#     print('train models with true density ratios')
    Myy_robust_list = []
    Myx_robust_list = []
    model_robust_list = []
    
    feature = xy[logging_index]
    eval_xshift = pdf_eval[logging_index]
    logging_xshift = pdf[logging_index]
    eval_policy = eval_policy[logging_index,logging_action]
    logging_policy = logging_policy[logging_index,logging_action]
    
    
    for i in range(n_class):
        ## generate training set for action i

        idx = (logging_action == i)


        data_i = feature[idx]
        reward_i = logging_reward[idx]
        eval_xshift_i = eval_xshift[idx]
        logging_xshift_i = logging_xshift[idx]
        eval_policy_i = eval_policy[idx]
        logging_policy_i = logging_policy[idx]

        train_size = len(data_i)

        if xshift_known:
            r = logging_xshift_i/eval_xshift_i
        if policy_known:
            w = logging_policy_i/eval_policy_i
        if use_xshift:
            w = r * w
        
        if dm:
            w = torch.ones(len(data_i))
        

        weight_st = my_bound(w)
        weighted_train = WEIGHT_DATA_SET(data_i,reward_i, w)

        train_model = Net()

        validate_size = int(0.1*train_size)

        validate_loader = data.DataLoader(data.Subset(weighted_train, range(0, validate_size)),
            batch_size=32, shuffle=True)
        # 10% validation set
        train_loader = data.DataLoader(data.Subset(weighted_train, range(validate_size, train_size)),
            batch_size=32, shuffle=True, )

        train_model, Myy, Myx, _, _, _ = train_validate_test(args,args.lr_robust, 50, "regression", 'cpu', 'False', train_model,
            train_loader, None , validate_loader, n_class , 0.000, d = n_dim,testflag = False)

        model_robust_list.append(train_model)
        Myy_robust_list.append(Myy)
        Myx_robust_list.append(Myx)
    return model_robust_list, Myy_robust_list, Myx_robust_list




In [86]:
def predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                        logging_action,\
                       xy,  pdf, pdf_eval, logging_policy, eval_policy,\
                       dm = False,xshift_known = True, policy_known = True,use_xshift = True):
    
    
    
    if xshift_known:
        r = pdf/pdf_eval
        
    if policy_known:
        w = logging_policy/eval_policy
#     print(pdf/pdf_eval)
#     print(logging_policy/eval_policy)
    if use_xshift:
        w = r*w
    if dm:
        w = torch.tensor([1])
    
    weight_st = my_bound(w)
    if len(xy.shape) == 2:
        output = model_robust_list[int(logging_action)](torch.tensor(xy))
    else:
        output = model_robust_list[int(logging_action)](torch.tensor(xy).unsqueeze(0))

    Myy = Myy_robust_list[int(logging_action)]
    Myx = Myx_robust_list[int(logging_action)]

    meanY, varY = ru.predict_regression(torch.tensor(weight_st), Myy, Myx, output, mean0, var0)
    return meanY.detach().numpy()
        
        
        

In [87]:
def evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            GT, model_robust_list, Myy_robust_list, Myx_robust_list,\
                                            dm = False,use_xshift = True):
    
#     model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
#                                             eval_index, eval_action, eval_reward,\
#                                             xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
#                                             dm=dm, use_xshift = use_xshift)
    
#     sample action 
    reg_loss = []
    triple_loss = []
    triple_r_loss = []
    sn = np.mean(eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action])
    
    tmp_pred = np.zeros((len(eval_index),n_class))
    for i in range(n_class):
        tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        i,xy[eval_index],  pdf[eval_index], pdf_eval[eval_index], \
                                        logging_policy[eval_index,i], eval_policy[eval_index,i],\
                                        dm = dm,use_xshift = use_xshift)
        tmp_eval = tmp_eval.reshape(-1,len(eval_index))
        tmp_pred[:,i] = tmp_eval
    
    
    regression_pred = np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
    triple = eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action] * (eval_reward - \
                                    tmp_pred[np.arange(len(eval_index)),eval_action]) + np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
    sn_triple = eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action]/sn * (eval_reward - \
                                    tmp_pred[np.arange(len(eval_index)),eval_action]) + np.sum(tmp_pred * eval_policy[eval_index,:],axis = 1)
    
                                                                                        

  


    return (np.mean(regression_pred)-GT)**2,(np.mean(triple)-GT)**2,(np.mean(sn_triple)-GT)**2



In [88]:
def evaluate_robust(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            GT, model_robust_list, Myy_robust_list, Myx_robust_list,\
                                            dm = False,use_xshift = True):
    
#     model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
#                                             eval_index, eval_action, eval_reward,\
#                                             xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
#                                             dm=dm, use_xshift = use_xshift)
    
#     sample action 
    reg_loss = []
    triple_loss = []
    triple_r_loss = []
    sn = np.mean(eval_policy[eval_index,eval_action]/logging_policy[eval_index,eval_action])
    for t in range(10):
        regression_list = []
        triple_r_list = []
        triple_list = []
        total_reward = 0
        for i in range(len(eval_index)):
            action, prob = take_action(eval_policy[eval_index[i]])
            rew = 0
            if action == label[eval_index[i]]:
                rew = 1
                total_reward += rew

            tmp_eval = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        action,xy[eval_index[i]],  pdf[eval_index[i]], pdf_eval[eval_index[i]], \
                                        logging_policy[eval_index[i],action], prob,\
                                        dm = dm,use_xshift = use_xshift)
            regression_list.append(tmp_eval)
            
            
            tmp_logging = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
                                        eval_action[i],xy[eval_index[i]],  pdf[eval_index[i]], pdf_eval[eval_index[i]], \
                                        logging_policy[eval_index[i],eval_action[i]], eval_policy[eval_index[i],eval_action[i]],\
                                        dm = dm,use_xshift = use_xshift)
            
            

            #     triple
            w = eval_policy[eval_index[i],eval_action[i]]/logging_policy[eval_index[i],eval_action[i]]
            triple_list.append(w*(eval_reward[i] - tmp_logging) + tmp_eval)
            triple_r_list.append(w/sn*(eval_reward[i] - tmp_logging) + tmp_eval)
            
            
            
            rw = my_bound(pdf_eval[eval_index[i]]/pdf[eval_index[i]] * w)
            
            
            
            
#             w = my_bound(eval_policy[eval_index[i],logging_action[i]]/logging_policy[logging_index[i],logging_action[i]])

#             tmp = predict_regression(model_robust_list, Myy_robust_list, Myx_robust_list,\
#                                         logging_action[i],xy[logging_index[i]], pdf[logging_index[i]], pdf_eval[logging_index[i]], \
#                                         logging_policy[logging_index[i],logging_action[i]], eval_policy[logging_index[i],logging_action[i]],\
#                                         dm = dm,use_xshift = use_xshift)
            
#             triple_r_list.append(rw*(logging_reward[i] - tmp) + tmp_eval)

#             triple_list.append(w*(logging_reward[i] - tmp) + tmp_eval)
 

        triple_r = np.mean(triple_r_list)
        triple = np.mean(triple_list)
        robust = np.mean(regression_list)
#         GT = total_reward/len(triple_r_list)
        reg_loss.append(robust)
        triple_loss.append(triple)
        triple_r_loss.append(triple_r)
    
    return (np.mean(reg_loss)-GT)**2,(np.mean(triple_loss)-GT)**2,(np.mean(triple_r_loss)-GT)**2



In [89]:
model_list = [
    'known ips',
    'known snips',
#     'known ips-r',
#     'unknown ips',
#     'unknown snips',
#     'unknown ips-r',
    'known dm',
    'known dmdr',
#     'known robust without xshift',
#     'known triple without xshift',
#     'known triple-r without xshift',
    'known robust',
    'known triple',
    'known sntriple'
#     'unknown dm',
#     'unknown dmdr',
#     'unknown dmdr-r',
#     'unknown robust without xshift',
#     'unknown triple without xshift',
#     'unknown triple-r without xshift',
#     'unknown robust',
#     'unknown triple',
#     'unknown triple-r'
]
len(model_list)

7

In [90]:
def estimate_GT_with_eval_policy(xy,label,eval_policy):
    eval_policy = eval_policy[train_size:]
    
    xy = xy[train_size:]
    
    label = label[train_size:]
    result = 0
    

    for i in range(len(xy)):
        result += eval_policy[i,label[i]]
    return result/len(eval_policy)
    
    

In [91]:
def return_dict_with_list(size_list):
    d = {}
    for i in range(len(size_list)):
        d[size_list[i]] = []
    return d

In [94]:
# ips
from tqdm import tqdm

# logging_a_list = [0.5,0.8,1.2]
# logging_b_list = [0.1,0.2,0.4]
size_list = [100,200,500,1000,2000]
train_size_list = [1000,2000,3000,5000,10000]
# train_size_list = [1000]
logging_policy_list = [[0.7,0.2],[0.5,0.2],[0.4,0.2],[0.4,0.3],[0.1,0.1]]
# logging_policy_list = [[0.5,0.2],[0.1,0]]


cc = 0    
frame = {}
frame['model'] = model_list

for bandit_train_size in train_size_list:
    for logging_interval in logging_policy_list:
        cc += 1
        print(cc)
        logging_policy, eval_policy = generate_policy_soften(logging_interval[0],logging_interval[1])

        GT = estimate_GT_with_eval_policy(xy,label,eval_policy)


        ips = return_dict_with_list(size_list)
        snips = return_dict_with_list(size_list)
        ips_r =return_dict_with_list(size_list)

        ips_unknown =return_dict_with_list(size_list)
        snips_unknown =return_dict_with_list(size_list)
        ips_r_unknown =return_dict_with_list(size_list)

        robust = return_dict_with_list(size_list)
        triple_noxshift = return_dict_with_list(size_list)
        triple = return_dict_with_list(size_list)
        sntriple_noxshift =return_dict_with_list(size_list)

        dm = return_dict_with_list(size_list)
        dmdr_noxshift = return_dict_with_list(size_list)
        dmdr = return_dict_with_list(size_list)

        robust_without_xshift = return_dict_with_list(size_list)
        triple_without_xshift = return_dict_with_list(size_list)
        triple_r_without_xshift = return_dict_with_list(size_list)

        pdf, mean_logging, std_logging = get_distribution(fc,logging_a, logging_b)
        pdf_eval, mean_eval, std_eval = get_distribution(fc,eval_a, eval_b)


        for _ in tqdm(range(10)):

            logging_index, logging_action, logging_reward = \
                    sample_logging_data(train_xy, train_label, bandit_train_size, pdf, pdf_eval, logging_policy, eval_policy)

            eval_index, eval_action, eval_reward = \
                        sample_eval_data(test_xy, test_label, 100, pdf, pdf_eval, logging_policy, eval_policy)  

            model_robust_list, Myy_robust_list, Myx_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            dm=False, use_xshift = True)
            model_dm_robust_list, Myy_dm_robust_list, Myx_dm_robust_list = train_robust_regression(logging_index, logging_action, logging_reward,\
                                            eval_index, eval_action, eval_reward,\
                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,\
                                            dm=True, use_xshift = True)


            for size in size_list:
                
                eval_index, eval_action, eval_reward = \
                        sample_eval_data(test_xy, test_label, size, pdf, pdf_eval, logging_policy, eval_policy)  
                res = evaluate_ips(logging_index, logging_action, logging_reward,\
                                 eval_index, eval_action, eval_reward,\
                                 xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT)
        
                ips[size].append(res[0])
                snips[size].append(res[1])


                ips_unknown[size].append(res[3])
                snips_unknown[size].append(res[4])



                res = evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                                            eval_index, eval_action, eval_reward,\
                                                            xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT,\
                                                            model_robust_list, Myy_robust_list, Myx_robust_list,\
                                                            dm = False,use_xshift = False)
                robust[size].append(res[0])
                triple_noxshift[size].append(res[1])
                sntriple_noxshift[size].append(res[2])




                res = evaluate_robust_all(logging_index, logging_action, logging_reward,\
                                    eval_index, eval_action, eval_reward,\
                                    xy,label,  pdf, pdf_eval, logging_policy, eval_policy,GT,\
                                    model_dm_robust_list, Myy_dm_robust_list, Myx_dm_robust_list,\
                                    dm = True,use_xshift = False)
                dm[size].append(res[0])
                dmdr_noxshift[size].append(res[1])
            for k in range(len(size_list)):
                size = size_list[k]
                policy_name = str(bandit_train_size) +'_'+ str(size_list[k]) + str(logging_interval)
                result_list = []
                
                

                result_list.append(np.sqrt(np.mean(ips[size])))
                result_list.append(np.sqrt(np.mean(snips[size])))

                result_list.append(np.sqrt(np.mean(dm[size])))
                result_list.append(np.sqrt(np.mean(dmdr_noxshift[size])))

                result_list.append(np.sqrt(np.mean(robust[size])))
                result_list.append(np.sqrt(np.mean(triple_noxshift[size])))
                result_list.append(np.sqrt(np.mean(sntriple_noxshift[size])))
                frame[policy_name] = result_list
frame = pd.DataFrame(frame).round(5)
print(frame)
# frame.to_csv('result2/newadv_noxshift.csv',index = False)


1


100%|███████████████████████████████████████████| 10/10 [00:45<00:00,  4.57s/it]


2


100%|███████████████████████████████████████████| 10/10 [00:48<00:00,  4.89s/it]


3


...

24


100%|███████████████████████████████████████████| 10/10 [01:56<00:00, 11.61s/it]


25


100%|███████████████████████████████████████████| 10/10 [01:56<00:00, 11.63s/it]

            model  1000_100[0.7, 0.2]  1000_200[0.7, 0.2]  1000_500[0.7, 0.2]  \
0       known ips             0.12545             0.13446             0.05460   
1     known snips             0.09190             0.05554             0.02647   
2        known dm             0.06269             0.07344             0.06830   
3      known dmdr             0.15331             0.10043             0.03137   
4    known robust             0.06820             0.06958             0.07341   
5    known triple             0.12365             0.08737             0.03175   
6  known sntriple             0.09924             0.06754             0.02874   

   1000_1000[0.7, 0.2]  1000_2000[0.7, 0.2]  1000_100[0.5, 0.2]  \
0              0.02791              0.03014             0.11518   
1              0.02413              0.01939             0.04624   
2              0.06715              0.07170             0.13283   
3              0.02694              0.01525             0.03924   
4              0

In [ ]:
frame

In [95]:
frame

model  1000_100[0.7, 0.2]  1000_200[0.7, 0.2]  1000_500[0.7, 0.2]  \
0       known ips             0.12545             0.13446             0.05460   
1     known snips             0.09190             0.05554             0.02647   
2        known dm             0.06269             0.07344             0.06830   
3      known dmdr             0.15331             0.10043             0.03137   
4    known robust             0.06820             0.06958             0.07341   
5    known triple             0.12365             0.08737             0.03175   
6  known sntriple             0.09924             0.06754             0.02874   

   1000_1000[0.7, 0.2]  1000_2000[0.7, 0.2]  1000_100[0.5, 0.2]  \
0              0.02791              0.03014             0.11518   
1              0.02413              0.01939             0.04624   
2              0.06715              0.07170             0.13283   
3              0.02694              0.01525             0.03924   
4              0.07116              0.06988             0.09488   
5              0.02676              0.01796             0.04047   
6              0.02636              0.01767             0.04001   

   1000_200[0.5, 0.2]  1000_500[0.5, 0.2]  1000_1000[0.5, 0.2]  ...  \
0             0.04850             0.05691              0.04797  ...   
1             0.05226             0.02029              0.01935  ...   
2             0.13202             0.12440              0.12830  ...   
3             0.05178             0.02222              0.02104  ...   
4             0.09628             0.09346              0.09844  ...   
5             0.05335             0.01915              0.02287  ...   
6             0.05052             0.01715              0.02248  ...   

   10000_100[0.4, 0.3]  10000_200[0.4, 0.3]  10000_500[0.4, 0.3]  \
0              0.08746              0.10025              0.05668   
1              0.05271              0.03255              0.01937   
2              0.11982              0.11314              0.11833   
3              0.04059              0.03527              0.02550   
4              0.11008              0.10459              0.10958   
5              0.03798              0.03805              0.02257   
6              0.04006              0.03534              0.02144   

   10000_1000[0.4, 0.3]  10000_2000[0.4, 0.3]  10000_100[0.1, 0.1]  \
0               0.02350               0.03188              0.25334   
1               0.02426               0.01402              0.10268   
2               0.11479               0.11511              0.31738   
3               0.01964               0.01135              0.14418   
4               0.10323               0.10424              0.29243   
5               0.02034               0.01213              0.13177   
6               0.02331               0.01154              0.10183   

   10000_200[0.1, 0.1]  10000_500[0.1, 0.1]  10000_1000[0.1, 0.1]  \
0              0.22480              0.09762               0.09448   
1              0.05301              0.02956               0.02800   
2              0.29502              0.31088               0.30642   
3              0.10330              0.05632               0.03248   
4              0.27915              0.29187               0.28787   
5              0.10441              0.04670               0.03335   
6              0.05857              0.02731               0.01358   

   10000_2000[0.1, 0.1]  
0               0.08345  
1               0.02863  
2               0.30842  
3               0.04736  
4               0.28808  
5               0.04356  
6               0.02771  

[7 rows x 126 columns]

In [93]:
pd.DataFrame(frame).round(5)

model  1000_100[0.7, 0.2]  1000_200[0.7, 0.2]  1000_500[0.7, 0.2]  \
0       known ips             0.16072             0.09458             0.02851   
1     known snips             0.08595             0.03730             0.01490   
2        known dm             0.06508             0.07889             0.07040   
3      known dmdr             0.10476             0.04527             0.01636   
4    known robust             0.05548             0.06463             0.05307   
5    known triple             0.11985             0.04882             0.01500   
6  known sntriple             0.09373             0.03758             0.01510   

   1000_1000[0.7, 0.2]  1000_2000[0.7, 0.2]  1000_100[0.5, 0.2]  \
0              0.03940              0.01619             0.09982   
1              0.02030              0.01412             0.05777   
2              0.06386              0.06955             0.10286   
3              0.02377              0.01084             0.04344   
4              0.05200              0.05686             0.11178   
5              0.02579              0.01078             0.06293   
6              0.02317              0.01070             0.05519   

   1000_200[0.5, 0.2]  1000_500[0.5, 0.2]  1000_1000[0.5, 0.2]  ...  \
0             0.09120             0.05712              0.02686  ...   
1             0.08243             0.03675              0.02342  ...   
2             0.10039             0.09464              0.09369  ...   
3             0.06753             0.02717              0.02205  ...   
4             0.11314             0.09927              0.10186  ...   
5             0.07496             0.03059              0.02195  ...   
6             0.07149             0.02926              0.02247  ...   

   1000_100[0.1, 0.1]  1000_200[0.1, 0.1]  1000_500[0.1, 0.1]  \
0             0.28366             0.23531             0.09231   
1             0.08286             0.06057             0.03610   
2             0.37510             0.37554             0.37270   
3             0.16582             0.11976             0.06115   
4             0.36224             0.35368             0.35463   
5             0.15452             0.11878             0.04954   
6             0.08687             0.05596             0.03954   

   1000_1000[0.1, 0.1]  1000_2000[0.1, 0.1]  2000_100[0.7, 0.2]  \
0              0.06585              0.04808             0.07760   
1              0.02631              0.01787             0.02338   
2              0.37250              0.37221             0.04888   
3              0.03818              0.02789             0.02810   
4              0.36135              0.35640             0.08770   
5              0.04867              0.02553             0.02961   
6              0.03688              0.01847             0.02893   

   2000_200[0.7, 0.2]  2000_500[0.7, 0.2]  2000_1000[0.7, 0.2]  \
0             0.02839             0.07269              0.03367   
1             0.05544             0.03016              0.00820   
2             0.03173             0.02460              0.02710   
3             0.04826             0.04866              0.01075   
4             0.09428             0.07965              0.08574   
5             0.04758             0.02640              0.01434   
6             0.04801             0.02667              0.01340   

   2000_2000[0.7, 0.2]  
0              0.02723  
1              0.00550  
2              0.02685  
3              0.00974  
4              0.08277  
5              0.00785  
6              0.00926  

[7 rows x 31 columns]

In [31]:
frame_len = frame.copy()

In [75]:
frame.to_csv('result2/mnist_result2.csv',index = False)

In [404]:
tmp_res

[0.0006503108584310573,
 1.5071289965870986e-05,
 0.00014912624994976813,
 0.0001966217316565533,
 0.002403711023301239,
 0.0003668544547247813,
 0.0003379121079380731]

In [405]:
result_list

[0.025501193274650058,
 0.018239821112019276,
 0.016477342001940082,
 0.01589913621870108,
 0.026133660873687365,
 0.02510542971825441,
 0.02425933609964079]

50000

In [ ]:
a = []
b = []
c = []
k = 0
while k < len(ips):
    a.append(ips[k])
    k += 1
    b.append(ips[k])
    k += 1
    c.append(ips[k])
    k += 1
print(np.sqrt(np.mean(a)),np.sqrt(np.mean(b)),np.sqrt(np.mean(c)))  

In [389]:
a = []
b = []
c = []
k = 0
while k < len(ips):
    a.append(snips[k])
    k += 1
    b.append(snips[k])
    k += 1
    c.append(snips[k])
    k += 1
print(np.sqrt(np.mean(a)),np.sqrt(np.mean(b)),np.sqrt(np.mean(c)))  

0.03102377622022736 0.020515294875127845 0.011759379455412883


In [392]:
a = []
b = []
c = []
k = 0
while k < len(ips):
    a.append(triple_noxshift[k])
    k += 1
    b.append(triple_noxshift[k])
    k += 1
    c.append(triple_noxshift[k])
    k += 1
print(np.sqrt(np.mean(a)),np.sqrt(np.mean(b)),np.sqrt(np.mean(c)))  

0.03073353914866272 0.018495907732385557 0.011262325168644553


In [394]:
a = []
b = []
c = []
k = 0
while k < len(ips):
    a.append(sntriple_noxshift[k])
    k += 1
    b.append(sntriple_noxshift[k])
    k += 1
    c.append(sntriple_noxshift[k])
    k += 1
print(np.sqrt(np.mean(a)),np.sqrt(np.mean(b)),np.sqrt(np.mean(c)))  

0.02971956963320808 0.01825370891149572 0.011315999262818321
